![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Hands-On Lab
# Welcome to the Governance Server Operation Lab

## Introduction

Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogs and platforms together so they can share information about data and technology (called metadata).

In this hands-on lab you will set up two types of governance server: an [Integration Daemon](https://egeria-project.org/concepts/integration-daemon/) and an [Engine Host](https://egeria-project.org/concepts/engine-host) that will work with a [Metadata Access Store](https://egeria-project.org/concepts/metadata-access-store/) to dynamically catalog metadata.  The aim is to show how these three types of [OMAG Servers](https://egeria-project.org/concepts/omag-server/) work together.

The Egeria team use the personas and scenarios from the fictitious company called [Coco Pharmaceuticals](https://egeria-project.org/practices/coco-pharmaceuticals/).
As part of the huge business transformation that Coco Pharmaceuticals has embarked on, they
have decided to use Egeria to manage their metadata across the enterprise.

**Note: this Lab needs Egeria release 3.15 or later to run**


## The Scenario

[Polly Tasker](https://egeria-project.org/practices/coco-pharmaceuticals/personas/polly-tasker/),
the lead of IT development asked [Peter Profile](https://egeria-project.org/practices/coco-pharmaceuticals/personas/peter-profile/) and [Erin Overview](https://egeria-project.org/practices/coco-pharmaceuticals/personas/erin-overview/) from the governance team to give a series of talks and a demos about Egeria to her team, who will be building integration connectors and governance action services to capture and govern their metadata using Egeria.  They are interested in understanding how these connectors run in Egeria so that they can test their work.

<figure style="margin-left: 7%; display:inline-block;">  
  <img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/polly-tasker.png">
  <figcaption style="margin-left: 7%;"><strong>Polly Tasker</strong></figcaption>
</figure>

<figure style="margin-left: 7%; display:inline-block;">  
  <img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/peter-profile.png">
  <figcaption style="margin-left: 7%;"><strong>Peter Profile</strong></figcaption>
</figure>

<figure style="margin-left: 7%; display:inline-block;">  
  <img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/erin-overview.png">
  <figcaption style="margin-left: 7%;"><strong>Erin Overview</strong></figcaption>
</figure>


Peter and Erin offer to demonstrate how to set up the runtime environment for these types of connectors and use various operational
commands to show their status.

They have been asked to set up the demo on the *Development [OMAG Server Platform](https://egeria-project.org/concepts/omag-server-platform/)* so that the development team can run it themselves at a later time.  Peter checks that the Development OMAG Server Platform is running.

----

In [ ]:
%run ../common/common-functions.ipynb

print(" ")
result = checkServerPlatform(devPlatformName, devPlatformURL)
print(" ")


----

If the platform is not running, follow [this link to set up and run the platform](https://egeria-project.org/education/open-metadata-labs/overview/).  Then re-run the previous step to ensure the platform is started.

The demo will focus on cataloguing files that are found in for following directory (folder).  It is simulating new data coming into the landing area of a data lake.  The files need to be catalogued to trigger provisioning processes that will move them to an appropriate place in the data lake.

-----

In [ ]:

landingAreaDirectory = fileSystemRoot + '/simple-demo/landing-area'


----

The data lake needs a metadata catalog.  Peter configures a [Metadata Access Store](https://egeria-project.org/concepts/metadata-access-store/) called *SimpleMetadataCatalog* on the Development OMAG Server Platform to act as the data lake catalog.  The *Metadata Access Store* is a type of server that they are familiar with after watching other demos from Peter and Erin including:

* The [Representing metadata](metadata-representations.ipynb) lab shows how different types of data are represented in the [Open Metadata Types](https://egeria-project.org/types/).
* The [Simple Cohort Demo](simple-cohort-demo.ipynb) lab shows how an [Open Metadata Repository Cohort](https://egeria-project.org/features/cohort-operation/overview/) works.

They do not spend much time on this piece.

----

In [ ]:

def configureMetadataCatalog(mdrServerName, mdrRepositoryType):
    eventBusURLroot   = os.environ.get('eventBusURLroot', 'localhost:9092')
    eventBusBody      = {
        "producer": {
             "bootstrap.servers": eventBusURLroot
         },
         "consumer": {
             "bootstrap.servers": eventBusURLroot
         }
    }
    print("Configuring " + mdrServerName + "...")
    configurePlatformURL(devPlatformURL, adminUserId, mdrServerName, devPlatformURL)
    configureMaxPageSize(devPlatformURL, adminUserId, mdrServerName, '600')
    clearServerType(devPlatformURL, adminUserId, mdrServerName)
    configureOwningOrganization(devPlatformURL, adminUserId, mdrServerName, "Coco Pharmaceuticals Dev Systems")
    configureUserId(devPlatformURL, adminUserId, mdrServerName, "simpleMDSnpa")
    configurePassword(devPlatformURL, adminUserId, mdrServerName, "simpleMDSpassw0rd")
    configureEventBus(devPlatformURL, adminUserId, mdrServerName, eventBusBody)
    configureMetadataRepository(devPlatformURL, adminUserId, mdrServerName, mdrRepositoryType)
    configureAccessService(devPlatformURL, adminUserId, mdrServerName, 'governance-engine', {})
    configureAccessService(devPlatformURL, adminUserId, mdrServerName, 'data-manager', {})
    configureAccessService(devPlatformURL, adminUserId, mdrServerName, 'asset-owner', {})
    configureAccessService(devPlatformURL, adminUserId, mdrServerName, 'asset-consumer', {})


simpleMetadataCatalog = "SimpleMetadataCatalog"

configureMetadataCatalog(simpleMetadataCatalog, "in-memory-repository")

print("\nDone.")


----

The developers know that they need to be able to write and test a variety of connectors.  They are particualr interested in *integration connectors* and *governance action services*

## Integration connectors

Erin begins with the [integration connectors](https://egeria-project.org/concepts/integration-connector/).  They sit on the edge of the open metadata ecosystem and exchange metadata between third party technologies and the open metadata ecosystem.  In the demo, Erin and Peter are going to show an integration connector called [Data Files Monitor Integration Connector](https://egeria-project.org/connectors/integration/data-files-monitor-integration-connector/) that detects files being added and deleted from a directory and maintains a catalog of the files in the Metadata Access Store.

This is the integration connector that is going to monitor the landing area directory.

## Connector providers and connector types

Every connector has a factory class called the [Connector Provider](/concepts/connector-provider).  This class is also able to return information about the capabilities of the connector in a connector type.  Erin shows the developers the [developer guide](https://egeria-project.org/guides/developer/integration-connectors/overview/) for integration connectors, pointing out the section on [building connector providers](https://egeria-project.org/guides/developer/integration-connectors/overview/#writing-the-connector-provider).  

The connector type from the connector provider provides valuable information to anyone configuring the connector into the Egeria runtime.
For example, the `recognizedConfigurationProperties` document which configuration properties can be set on the connector's connection object.

This is the connector type from the *Data Files Monitor Integration Connector*'s provider:

----

In [ ]:
dataFilesMonitorProviderClassName = "org.odpi.openmetadata.adapters.connectors.integration.basicfiles.DataFilesMonitorIntegrationProvider"

getConnectorType(devPlatformName, devPlatformURL, dataFilesMonitorProviderClassName)
   

----

Notice that one of the connector's implemented interfaces is `org.odpi.openmetadata.integrationservices.files.connector.`**FilesIntegratorOMISConnector**.

## Integration daemons 

Integration connectors run in an [Integration Daemon](https://egeria-project.org/concepts/integration-daemon/).  Like the *Metadata Access Store*, the *Integration Daemon* is a type of [OMAG Server](https://egeria-project.org/concepts/omag-server/) that runs in the *OMAG Server Platform*.  In this demo an integration daemon called *SimpleIntegrationDaemon* will run on the Development OMAG Server Platform with the *SimpleMetadataCatalog*.

<center>
    <img src="../images/governance-engine-lab-platform-part.png">
</center>

Integration daemons have registered services called Open Metadata Integration Services (OMISs).  An OMIS provides a specialist API called the *context* to an integration connector that makes it easy to work with metadata for a paticular catagory of third party technology.

These are the OMISs available on the Development OMAG Server Platform.

----

In [ ]:

getIntegrationServices(devPlatformName, devPlatformURL)


----

## Configuring an integration connector in Egeria

Peter sets up *Data Files Monitor Integration Connector* to monitor a landing area directory.  It is configured to run in *SimpleIntegrationDaemon*.  Because it is working with files, it uses the *Files Integrator OMIS*.  It is confirmed by the connector interface it implemented called:

`org.odpi.openmetadata.integrationservices.files.connector.`**FilesIntegratorOMISConnector**.

The configuration defines a list of integration connectors that are going to run in the integration daemon.  In this demo, there is only one integration connector so the configuration is quite simple.

----

In [ ]:
def configureIntegrationDaemon(idServerName, mdrServerName, integrationService, connectorConfigs):
    eventBusURLroot   = os.environ.get('eventBusURLroot', 'localhost:9092')
    eventBusBody      = {
        "producer": {
             "bootstrap.servers": eventBusURLroot
         },
         "consumer": {
             "bootstrap.servers": eventBusURLroot
         }
    }
    print("Configuring " + idServerName + "...")
    configurePlatformURL(devPlatformURL, adminUserId, idServerName, devPlatformURL)
    configureMaxPageSize(devPlatformURL, adminUserId, idServerName, '600')
    clearServerType(devPlatformURL, adminUserId, idServerName)
    configureOwningOrganization(devPlatformURL, adminUserId, idServerName, "Coco Pharmaceuticals Dev Systems")
    configureUserId(devPlatformURL, adminUserId, idServerName, "simplenpa")
    configurePassword(devPlatformURL, adminUserId, idServerName, "simplepassw0rd")
    configureEventBus(devPlatformURL, adminUserId, idServerName, eventBusBody)
    configureDefaultAuditLog(devPlatformURL, adminUserId, idServerName)
    configureIntegrationService(devPlatformURL, adminUserId, idServerName, mdrServerName, devPlatformURL, integrationService, {}, connectorConfigs)


simpleIntegrationDaemon = "SimpleIntegrationDaemon"

DataFilesMonitorConnectorName       = "LandingAreaFilesMonitor"

DataFilesMonitorConnectorConnection = { 
                                      "class" : "Connection",
                                      "connectorType" : 
                                      {
                                           "class" : "ConnectorType",
                                           "connectorProviderClassName" : "org.odpi.openmetadata.adapters.connectors.integration.basicfiles.DataFilesMonitorIntegrationProvider"           
                                      },
                                      "endpoint" :
                                      {
                                           "class" : "Endpoint",
                                           "address" : landingAreaDirectory
                                      }
                                }

connectorConfigs = [ 
    {
        "class"                       : "IntegrationConnectorConfig",
        "connectorName"               : DataFilesMonitorConnectorName,
        "connectorUserId"             : "simplenpa",
        "connection"                  : DataFilesMonitorConnectorConnection,
        "metadataSourceQualifiedName" : "SimpleLandingArea",
        "refreshTimeInterval"         : 10,
        "usesBlockingCalls"           : "false"
    }]



configureIntegrationDaemon(simpleIntegrationDaemon, simpleMetadataCatalog, "files-integrator", connectorConfigs)

print ("\nDone.")



----

Peter then starts both servers on the platform.

When *SimpleIntegrationDaemon* starts, it connects to *SimpleMetadataCatalog* and initalizes the *Data Files Monitor Integration Connector*.

<center>
    <img src="../images/governance-engine-lab-servers-part.png">
</center>

----

In [ ]:
activatePlatform(devPlatformName, devPlatformURL, [simpleMetadataCatalog, simpleIntegrationDaemon])

print("\nDone.")


----

Peter checks that the servers are running on the platform ...

----

In [ ]:
queryActiveServers(devPlatformName, devPlatformURL)

----

He checks to see if any files have been catalogued:

----

In [ ]:

assetOwnerPrintAssets(simpleMetadataCatalog, devPlatformName, devPlatformURL, petersUserId, ".*landing.*")


----

Peter then queries the status of *SimpleIntegrationDaemon* to be sure the integration connector is started.

----

In [ ]:

getIntegrationDaemonStatus(simpleIntegrationDaemon, devPlatformName, devPlatformURL, petersUserId)


----

*Data Files Monitor Integration Connector* has started but it is in `FAILED` status because the landing area directory it is supposed monitor does not exist.

This is expected ... the plan is to use a [Governance Action Service](https://egeria-project.org/concepts/governance-action-service/) called
[Move/Copy File Governance Action Service](https://egeria-project.org/connectors/governance-action/move-copy-file-provisioning-governance-action-service/)
to create the directory by provisioning files into it.  The files will come from the sample data in Egeria's install image:

----

In [ ]:

filesSourceFolder = egeriaSampleDataRoot + '/sample-data/oak-dene-drop-foot-weekly-measurements'


----

Governance action services are also connectors so *Move/Copy File Governance Action Service* has a connector provider
and it is possible to query its connector type.

----

In [ ]:

moveCopyFileProviderClassName = "org.odpi.openmetadata.adapters.connectors.governanceactions.provisioning.MoveCopyFileGovernanceActionProvider"

getConnectorType(devPlatformName, devPlatformURL, moveCopyFileProviderClassName)


----

Notice that one of the connector's implemented interfaces is `org.odpi.openmetadata.frameworks.governanceaction.`**GovernanceActionService**.


Governance action services are often more configurable and so you see a longer list of recognized configuration properties.  You can look up the meaning of these properties in the [connector catalog](https://egeria-project.org/connectors/governance-action/move-copy-file-provisioning-governance-action-service/).


## Configuring governance action services in Egeria

Governance action services run in [Governance Action Engines](https://egeria-project.org/concepts/governance-action-engine/).  Governance action engines are a type of [governance engine](https://egeria-project.org/concepts/governance-engine/) and are hosted in [Engine Host](https://egeria-project.org/concepts/engine-host) OMAG Servers.

The engine host has registered services called [Open Metadata Engine Services (OMESs)](https://egeria-project.org/services/omes/).  Each OMES runs a different type of governance engine.

This is the full list of the OMESs available on the Development OMAG Server Platform.  

----

In [ ]:

getEngineServices(devPlatformName, devPlatformURL)


----

Since *Move/Copy File Governance Action Service* is a governance action service, it runs under the under the [Governance Action OMES](https://egeria-project.org/services/omes/governance-action/overview).

Peter configures an engine host called *SimpleEngineHost* that is running a governance action engine called *SimpleGovernanceEngine*.  

*SimpleEngineHost* runs on the Development OMAG Server Platform with *SimpleMetadataCatalog* and *SimpleIntegrationDaemon*.

----

In [ ]:

def configureSimpleEngineHost(engineServerName, mdrServerName, engineService, engineConfigs):
    eventBusURLroot   = os.environ.get('eventBusURLroot', 'localhost:9092')
    eventBusBody      = {
        "producer": {
             "bootstrap.servers": eventBusURLroot
         },
         "consumer": {
             "bootstrap.servers": eventBusURLroot
         }
    }
    print("Configuring " + engineServerName + "...")
    configurePlatformURL(devPlatformURL, adminUserId, engineServerName, devPlatformURL)
    configureMaxPageSize(devPlatformURL, adminUserId, engineServerName, '600')
    clearServerType(devPlatformURL, adminUserId, engineServerName)
    configureOwningOrganization(devPlatformURL, adminUserId, engineServerName, "Coco Pharmaceuticals Dev Systems")
    configureUserId(devPlatformURL, adminUserId, engineServerName, "simplenpa")
    configurePassword(devPlatformURL, adminUserId, engineServerName, "simplepassw0rd")
    configureEventBus(devPlatformURL, adminUserId, engineServerName, eventBusBody)
    configureDefaultAuditLog(devPlatformURL, adminUserId, engineServerName)
    configureEngineDefinitionServices(devPlatformURL, adminUserId, engineServerName, mdrServerName, devPlatformURL)
    configureGovernanceEngineService(devPlatformURL, adminUserId, engineServerName, mdrServerName, devPlatformURL, engineService, engineConfigs)


simpleEngineHost = "SimpleEngineHost"
simpleGovernanceEngine = "SimpleGovernanceEngine"

governanceActionEngines = [ 
    {
        "class"               : "EngineConfig",
        "engineQualifiedName" : simpleGovernanceEngine,
        "engineUserId"        : "simplenpa"
    }]

configureSimpleEngineHost(simpleEngineHost, simpleMetadataCatalog, "governance-action", governanceActionEngines)

print ("\nDone.")



----

Peter then starts up *SimpleEngineHost*.  It connects to *SimpleMetadataCatalog* and initializes the *SimpleGovernanceEngine*.

<center>
    <img src="../images/governance-engine-lab-servers-part-2.png">
</center>

----

In [ ]:
activatePlatform(devPlatformName, devPlatformURL, [simpleEngineHost])

print("\nDone.")


----

There are now three servers running on the Development OMAG Server Platform: *SimpleEngineHost*, *SimpleMetadataCatalog* and *SimpleIntegrationDaemon*.

<center>
    <img src="../images/governance-engine-lab-platform.png">
</center>


----

In [ ]:
queryActiveServers(devPlatformName, devPlatformURL)

----

Peter queries the status of *SimpleEngineHost* to check that *SimpleGovernanceEngine* has started.

----

In [ ]:

printGovernanceEngineStatuses(simpleEngineHost, devPlatformName, devPlatformURL, petersUserId)


----

*SimpleGovernanceEngine* has started, but it is status is `ASSIGNED` which means it is waiting for information from *SimpleMetadataCatalog*.  

The definition of which governance action services are supported by a governance action engine is managed as an open metadata definition rather than hard-coded in the server configuration.  This is to allow the definition of the governance engine to be shared between engine hosts.  New governance action services can also be introduced without needing to restart the engine host.  

At this point in time, *SimpleEngineHost* has been unable to retrieve the definition of *SimpleGovernanceEngine* from *SimpleMetadataCatalog*.

Governance engine definitions are typically delivered through an [Open Metadata Archive](https://egeria-project.org/concepts/governance-engine-pack/) because they include multiple related elements and the archive makes it easy to maintain and distribute.  Governance engine definitions can also be added dynamically through *SimpleMetadataCatalog*'s REST API.  Erin said they would use the REST API because it is a useful way to understand how the engine host works.

The plan is to gradually build up the definition for *SimpleGovernanceEngine* in *SimpleMetadataCatalog* until the
*SimpleGovernanceEngine*'s status is `RUNNING`.  The first step is to add the root governance engine definition
to *SimpleMetadataCatalog*.  

----

In [ ]:
governanceEngineDisplayName = "Simple Governance Engine"
governanceEngineDescription = "Supports the simple governance servers lab."

governanceEngineGUID = createGovernanceEngine(simpleMetadataCatalog,
                                              devPlatformName,
                                              devPlatformURL,
                                              petersUserId,
                                              "GovernanceActionEngine",
                                              simpleGovernanceEngine,
                                              governanceEngineDisplayName,
                                              governanceEngineDescription)

if (governanceEngineGUID):
    print (" ")
    print ("The guid for the " + simpleGovernanceEngine + " governance engine is: " + governanceEngineGUID)
    print (" ")

print ("Done. ")    

----

Now the governance engine is defined, its definition is sent to *SimpleEngineHost* in an event
and the engine's status moves to `CONFIGURING`.
Notice more descriptive information is returned with the status.  This is coming from the new governance engine definition.

----

In [ ]:

waitForConfiguringGovernanceEngine(simpleEngineHost, devPlatformName, devPlatformURL, petersUserId, simpleGovernanceEngine)

printGovernanceEngineStatuses(simpleEngineHost, devPlatformName, devPlatformURL, petersUserId)


----

Now we add the description of *Move/Copy File GovernanceAction Service* to *SimpleMetadataCatalog*.  There are two parts to
registering a governance action service.  The first is to create a GovernanceService definition that identifies
the implementation of the governance action service and the second part registers this
GovernanceService definition with the GovernanceEngine definition.
This registration maps one or more of the governance engine's request types, along with the default request parameters to
the GovernanceService definition.  This mapping is used to translate a request to the governance engine into an
invocation of a governance action service.

This picture shows the structure of the resulting definitions for a governance engine.  A governance action service may be
registered with multiple governance engines, using the same or different request types. 
It can also be registered multiple time with different request types.

![logical governance engine definition](../images/governance-action-request-type.png)
> Logical structure of the governance services within a governance engine

This next code issues the calls to create a definition of the governance action service and add it to the definition for *SimpleGovernanceEngine*.

----

In [ ]:

ftpGovernanceServiceName = "ftp-governance-action-service"
ftpGovernanceServiceDisplayName = "FTP Governance Action Service"
ftpGovernanceServiceDescription = "Simulates FTP from an external party."
ftpGovernanceServiceProviderClassName = moveCopyFileProviderClassName
ftpGovernanceServiceConfigurationProperties = {
        "noLineage" : ""
    }
ftpGovernanceServiceRequestType = "copy-file"




governanceServiceGUID = createGovernanceService(simpleMetadataCatalog,
                                                devPlatformName,
                                                devPlatformURL,
                                                erinsUserId,
                                                "GovernanceActionService",
                                                ftpGovernanceServiceName,
                                                ftpGovernanceServiceDisplayName,
                                                ftpGovernanceServiceDescription,
                                                ftpGovernanceServiceProviderClassName,
                                                ftpGovernanceServiceConfigurationProperties)

if (governanceServiceGUID):
    print (" ")
    print ("The guid for the " + ftpGovernanceServiceName + " governance service is: " + governanceServiceGUID)
    registerGovernanceServiceWithEngine(simpleMetadataCatalog,
                                        devPlatformName,
                                        devPlatformURL,
                                        petersUserId,
                                        governanceEngineGUID,
                                        governanceServiceGUID,
                                        ftpGovernanceServiceRequestType,
                                        ftpGovernanceServiceRequestType)
    print ("Service registered as: " + ftpGovernanceServiceRequestType)
    print (" ")
    

----

*Move/Copy File Governance Action Service* is defined in *SimpleGovernanceEngine* and it is ready to be called.

<center>
    <img src="../images/governance-engine-lab-servers.png">
</center>

When at least one governance service is registered with the governance engine, the status of the governance engine in
`governDL01` moves to `RUNNING` and it is possible to see the list of supported **request types** for the governance engine. 

----

In [ ]:

waitForRunningGovernanceEngine(simpleEngineHost, devPlatformName, devPlatformURL, petersUserId, simpleGovernanceEngine)

printGovernanceEngineStatuses(simpleEngineHost, devPlatformName, devPlatformURL, petersUserId)


----

Any defined governance action service can be called directly through the Governance Engine OMAS,
which in this lab is running in *SimpleMetadataCatalog*.  This call results in an event being sent to all
engine hosts running *SimpleGovernanceEngine*.  The first one that claims it gets to run it.  In this lab
there is only one engine host and so the request will run on *SimpleEngineHost*.

This governance action does not take long to run so you should soon see its status as completed.

----

In [ ]:

def addFileToLandingArea(sourceFolder, destinationFolder, weekNumber, qualifiedName):
    sourceFileName = sourceFolder + "/" + "week" + weekNumber + ".csv"
    requestParameters = {
        "sourceFile" : sourceFileName,
        "destinationFolder" : destinationFolder
    }
    governanceActionGUID = None
    governanceActionGUID = initiateGovernanceAction(simpleMetadataCatalog,
                                                    devPlatformName,
                                                    devPlatformURL,
                                                    erinsUserId,
                                                    simpleGovernanceEngine,
                                                    qualifiedName,
                                                    ftpGovernanceServiceRequestType,
                                                    requestParameters,
                                                    "Governance Services Lab")
    waitForRunningGovernanceAction(simpleMetadataCatalog, devPlatformName, devPlatformURL, erinsUserId, governanceActionGUID)
    printGovernanceAction(simpleMetadataCatalog, devPlatformName, devPlatformURL, erinsUserId, governanceActionGUID)

    
addFileToLandingArea(filesSourceFolder, landingAreaDirectory, "1", "FTP Oak Dene Week 1")

----
We now have the ability to provision new files into the landing area.

Restarting the integration connector in *SimpleIntegrationDaemon* will cause it to
test again to see if the directory it is monitoring exists.

----

In [ ]:

restartIntegrationConnector(simpleIntegrationDaemon, devPlatformName, devPlatformURL, petersUserId, DataFilesMonitorConnectorName)


----

If we check the status of the integration connectors again, you can see this connector is now running.

----

In [ ]:

getIntegrationDaemonStatus(simpleIntegrationDaemon, devPlatformName, devPlatformURL, petersUserId)


----

The connector will begin cataloguing any files placed in the directory.  We should be able to see the `week1.csv` file in the catalog along with catalog entries for each of the nested directories it sits in. If no assets are shown wait a short while and run the cell again and they will appear.

----

In [ ]:

assetOwnerPrintAssets(simpleMetadataCatalog, devPlatformName, devPlatformURL, petersUserId, ".*landing.*")


----
The action of the integration connector has been to catalog the new file and the folders above it.
The catalog definition is minimal, consisting of just what can be gleaned from the file system.

It is possible to provide the integration connector with a template metadata element to copy when it
is cataloguing files.   This template can include classifications such as zones, origin, confidentiality and attachments such as
connections, schemas and lineage mappings. (There is more information on templates in the
[Egeria website](https://egeria-project.org/features/templated-cataloguing/overview/)).

Templates are used to set up values for the assets created by the integration connector that are always the same.
In this example, all of the files that this connector encounters are from the same source so we can use a template to set up the
file's origin and the connection information needed to access the file.

Including this type of detail in the asset means that no-one has to remember to add governancem etadata to the new assets once they are catalogued.
Other values that are useful to set up in a template are any licenses for the file, schema information, zones, known lineage to this
directory, plus other classifications.  Information about the types of information that can be attached to an
asset are available on the [Egeria website](https://egeria-project.org/patterns/metadata-manager/overview/).

## Working with templates

If you know the class name of an integration connector's provider, it is possible to check if the connector is of the right type for an integration service.  This function also returns full details of the connector type, which often includes descriptive information as well as the configuration properties that it supports.

The command below request that the Files Integrator OMIS service validate and return information about the configured connector.  It includes the connector type that was returned by the platform plus mor information that is available for an integration connector.

----

In [ ]:

print("Data Files Monitor Integration Connector Report:")

validateIntegrationConnector(simpleIntegrationDaemon, devPlatformName, devPlatformURL, "files-integrator", petersUserId, dataFilesMonitorProviderClassName)


----
This connector supports the `templateQualifiedName` and the `allowCatalogDelete` configuration properties.  If you are curious
about their meaning, review the definitions in the [connector catalog](https://egeria-project.org/connectors/integration/data-files-monitor-integration-connector/)

We are going to set the `templateQualifiedName` with the qualified name of an asset that has the origin set up for the each of the appropriate originating hospitals and membership of the default governance zone of "quarantine".
This zone means that the asset is still being set up and it is not visible to the data lake users.

The commands below create the template asset for the landing area. 

----

In [ ]:
newAssetZone    = "quarantine"

t1AssetName     = "Landing Area Template"
t1QualifiedName = "template:simple-demo"
t1DisplayName   = "Simple Demo Template Asset"
t1Description   = "Template asset for Landing Area."
t1Contact       = "Simple Contact Person"
t1Dept          = "Simple Research Centre"
t1Org           = "Simple Hospital"

template1guids = assetOwnerCreateCSVAsset(simpleMetadataCatalog, devPlatformName, devPlatformURL, petersUserId, t1DisplayName, t1Description, t1QualifiedName)
template1guid = getLastGUID(template1guids)
addOrigin(simpleMetadataCatalog, devPlatformName, devPlatformURL, petersUserId, t1AssetName, template1guid, t1Contact, t1Dept, t1Org)
addZones(simpleMetadataCatalog, devPlatformName, devPlatformURL, petersUserId, t1AssetName, template1guid, [newAssetZone])


print ("\n\nNewTemplate Assets:")
assetOwnerPrintAssets(simpleMetadataCatalog, devPlatformName, devPlatformURL, petersUserId, ".*Template.*")

----

The next commands reconfigure the integration connector that is monitoring for new files in the landing area to use the template ...

---

In [ ]:
t1ConfigurationProperties = {
    "templateQualifiedName" : t1QualifiedName
}


updateConnectorConfigurationProperties(simpleIntegrationDaemon, devPlatformName, devPlatformURL, petersUserId, DataFilesMonitorConnectorName, t1ConfigurationProperties)
getIntegrationConnectorConfigProperties(simpleIntegrationDaemon, devPlatformName, devPlatformURL, petersUserId, DataFilesMonitorConnectorName)

In [ ]:
assetOwnerPrintAssets(simpleMetadataCatalog, devPlatformName, devPlatformURL, petersUserId, ".*landing.*")

----

Now lets add the file for week 2 into the landing area.

----

In [ ]:

addFileToLandingArea(filesSourceFolder, landingAreaDirectory, "2", "FTP Oak Dene Week 2")
 

----

The newly catalogued file will show that the origin and zones are set up.

----

In [ ]:

assetOwnerPrintAssets(simpleMetadataCatalog, devPlatformName, devPlatformURL, petersUserId, ".*landing-.*")


----

## Deleting files

The finale of this lab is to show that the integration connector must also remove metadata for resources (files) that no longer exist.

The code below adds another variations of the *Move/Copy File Governance Action Service* that will delete a file.

----

In [ ]:
dlGovernanceServiceName = "delete-files-governance-action-service"
dlGovernanceServiceDisplayName = "File Delete Governance Action Service"
dlGovernanceServiceDescription = "Removes weekly measurement files from the landing area."
dlGovernanceServiceProviderClassName = moveCopyFileProviderClassName
dlGovernanceServiceConfigurationProperties = {
        "noLineage" : ""
    }
dlGovernanceServiceRequestType = "delete-file"

governanceServiceGUID = createGovernanceService(simpleMetadataCatalog,
                                                devPlatformName,
                                                devPlatformURL,
                                                erinsUserId,
                                                "GovernanceActionService",
                                                dlGovernanceServiceName,
                                                dlGovernanceServiceDisplayName,
                                                dlGovernanceServiceDescription,
                                                dlGovernanceServiceProviderClassName,
                                                dlGovernanceServiceConfigurationProperties)

if (governanceServiceGUID):
    print (" ")
    print ("The guid for the " + dlGovernanceServiceName + " governance service is: " + governanceServiceGUID)
    registerGovernanceServiceWithEngine(simpleMetadataCatalog,
                                        devPlatformName,
                                        devPlatformURL,
                                        petersUserId,
                                        governanceEngineGUID,
                                        governanceServiceGUID,
                                        dlGovernanceServiceRequestType,
                                        dlGovernanceServiceRequestType)
    print ("Service registered as: " + dlGovernanceServiceRequestType)
    print (" ")

----

This is the resulting status of *SimpleGovernanceEngine*.  Notice that is now supports request types of **copy-file** and **delete-file**.

----

In [ ]:

printGovernanceEngineStatuses(simpleEngineHost, devPlatformName, devPlatformURL, petersUserId)


----

The call below removes the first file added to the landing area.

----

In [ ]:
def removeFileFromLandingArea(sourceFolder, weekNumber, qualifiedName):
    sourceFileName = sourceFolder + "/" + "week" + weekNumber + ".csv"
    requestParameters = {
        "sourceFile" : sourceFileName
    }
    governanceActionGUID = None
    governanceActionGUID = initiateGovernanceAction(simpleMetadataCatalog,
                                                    devPlatformName,
                                                    devPlatformURL,
                                                    erinsUserId,
                                                    simpleGovernanceEngine,
                                                    qualifiedName,
                                                    dlGovernanceServiceRequestType,
                                                    requestParameters,
                                                    "Governance Services Lab")
    waitForRunningGovernanceAction(simpleMetadataCatalog, devPlatformName, devPlatformURL, erinsUserId, governanceActionGUID)
    printGovernanceAction(simpleMetadataCatalog, devPlatformName, devPlatformURL, erinsUserId, governanceActionGUID)



removeFileFromLandingArea(landingAreaDirectory, "1", "Remove Oak Dene Week 1")


----

Erin queries the catalogued files to show that week 1 has been deleted.

----

In [ ]:
assetOwnerPrintAssets(simpleMetadataCatalog, devPlatformName, devPlatformURL, erinsUserId, ".*landing-.*")

----

Finally, Peter displays the governance actions that have run in *SimpleEngineHost*.  The request is to *SimpleMetadataCatalog* because, in addition to maintaining and distributing the governance engine definitions, the *SimpleMetadataCatalog* maintains the status and outcomes of each governance action.

----

In [ ]:
monitorGovernanceActions(simpleMetadataCatalog, devPlatformName, devPlatformURL, petersUserId)

----

## Summary

This demo has showed how to configure the Integration Daemon and Engine Host governance servers with specialized connectors written to automate metadata maintenance and governance.


----
## Where to next

* [Automated Curation](../asset-management-labs/automated-curation.ipynb) - follow Erin and Peter as they build a comprehensive onboarding pipeline by combining integration connectors with governance action services in a [Governance Action Process](https://egeria-project.org/concepts/governance-action-process/).

----